In [22]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [23]:
# load in csv 

df = pd.read_csv('data/nfl_games.csv')

df.tail(5)

,date,season,neutral,playoff,team1,team2,elo1,elo2,elo_prob1,score1,score2,result1
15735,1/15/2017,2016,0,1,DAL,GB,1617.794683,1635.451172,0.567714,31,34,0.0
15736,1/15/2017,2016,0,1,KC,PIT,1681.926463,1647.734179,0.638993,16,18,0.0
15737,1/22/2017,2016,0,1,ATL,GB,1664.127266,1651.537731,0.609840,44,21,1.0
15738,1/22/2017,2016,0,1,NE,PIT,1747.160321,1662.437215,0.703052,36,17,1.0
15739,2/5/2017,2016,1,1,ATL,NE,1688.081418,1763.818168,0.392700,28,34,0.0


In [24]:
# check datatype
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15740 entries, 0 to 15739
Data columns (total 12 columns):
date         15740 non-null object
season       15740 non-null int64
neutral      15740 non-null int64
playoff      15740 non-null int64
team1        15740 non-null object
team2        15740 non-null object
elo1         15740 non-null float64
elo2         15740 non-null float64
elo_prob1    15740 non-null float64
score1       15740 non-null int64
score2       15740 non-null int64
result1      15740 non-null float64
dtypes: float64(4), int64(5), object(3)
memory usage: 1.4+ MB


In [25]:
# limit data to the previous 3 seasons & non-playoff (regular season) games
df = df[df['season']>2013]
df = df[df['playoff']==0]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 768 entries, 14939 to 15728
Data columns (total 12 columns):
date         768 non-null object
season       768 non-null int64
neutral      768 non-null int64
playoff      768 non-null int64
team1        768 non-null object
team2        768 non-null object
elo1         768 non-null float64
elo2         768 non-null float64
elo_prob1    768 non-null float64
score1       768 non-null int64
score2       768 non-null int64
result1      768 non-null float64
dtypes: float64(4), int64(5), object(3)
memory usage: 78.0+ KB


In [26]:
# check for empty entries
df.isnull().sum()

date         0
season       0
neutral      0
playoff      0
team1        0
team2        0
elo1         0
elo2         0
elo_prob1    0
score1       0
score2       0
result1      0
dtype: int64

In [27]:
# Add datetime column

df['datetime'] = pd.to_datetime(df['date'])

# Add elo_prob2 column

df['elo_prob2'] = 1-pd.to_numeric(df['elo_prob1'])

# parse dataset by team, elo scores, and elo probabilities

split_t1 = df[['date','datetime','season','team1','elo1','elo_prob1']]
split_t2 = df[['date','datetime','season','team2','elo2','elo_prob2']]

split_t1.columns = ['date','datetime','season','team','elo','probability']
split_t2.columns = ['date','datetime','season','team','elo','probability']

merge_df = [split_t1,split_t2]
merged = pd.concat(merge_df)

In [28]:
merged['team'].unique()

array(['SEA', 'PHI', 'PIT', 'BAL', 'TB', 'DEN', 'LAR', 'ATL', 'CHI', 'DAL',
       'HOU', 'KC', 'MIA', 'NYJ', 'ARI', 'DET', 'BUF', 'CAR', 'CIN', 'WSH',
       'CLE', 'GB', 'MIN', 'SF', 'NYG', 'LAC', 'TEN', 'OAK', 'IND', 'JAX',
       'NO', 'NE'], dtype=object)

In [32]:
# add conference and division
afc_teams =['BUF','MIA','NE','NYJ',
            'BAL','CIN','CLE','PIT',
            'HOU','IND','JAX','TEN',
            'DEN','KC','LAC','OAK']
nfc_teams =['DAL','NYG','PHI','WSH',
            'CHI','DET','GB','MIN',
            'ATL','CAR','NO','TB',
            'ARI','LAR','SF','SEA']
afceast = ['BUF','MIA','NE','NYJ']
afcnorth = ['BAL','CIN','CLE','PIT']
afcsouth = ['HOU','IND','JAX','TEN']
afcwest = ['DEN','KC','LAC','OAK']
nfceast = ['DAL','NYG','PHI','WSH']
nfcnorth = ['CHI','DET','GB','MIN']
nfcsouth = ['ATL','CAR','NO','TB']
nfcwest = ['ARI','LAR','SF','SEA']

merged['conference'] = np.where(merged['team'].isin(afc_teams), 'AFC',
                                np.where(merged['team'].isin(nfc_teams), 'NFC',''))
merged['division'] = np.where(merged['team'].isin(afceast), 'AFC East',(
                        np.where(merged['team'].isin(afcnorth), 'AFC North',
                           np.where(merged['team'].isin(afcsouth), 'AFC South',
                                np.where(merged['team'].isin(afcwest), 'AFC West',
                                    np.where(merged['team'].isin(nfceast), 'NFC East',
                                        np.where(merged['team'].isin(nfcnorth), 'NFC North',
                                            np.where(merged['team'].isin(nfcsouth), 'NFC South',
                                                np.where(merged['team'].isin(nfcwest), 'NFC West','')))))))))

                        

In [34]:
merged.sort_values(['season','datetime','probability'],ascending=[True,True,False])
merged.reset_index(inplace=True, drop=True)
merged

,date,datetime,season,team,elo,probability,conference,division
0,9/4/2014,2014-09-04,2014,SEA,1678.971000,0.803040,NFC,NFC West
1,9/7/2014,2014-09-07,2014,PHI,1528.197000,0.791858,NFC,NFC East
2,9/7/2014,2014-09-07,2014,PIT,1537.231000,0.772161,AFC,AFC North
3,9/7/2014,2014-09-07,2014,BAL,1529.779000,0.549330,AFC,AFC North
4,9/7/2014,2014-09-07,2014,TB,1436.161000,0.392988,NFC,NFC South
5,9/7/2014,2014-09-07,2014,DEN,1624.090000,0.702028,AFC,AFC West
6,9/7/2014,2014-09-07,2014,LAR,1512.750000,0.643936,NFC,NFC West
7,9/7/2014,2014-09-07,2014,ATL,1458.825000,0.428661,NFC,NFC South
8,9/7/2014,2014-09-07,2014,CHI,1493.573000,0.643228,NFC,NFC North
9,9/7/2014,2014-09-07,2014,DAL,1501.041000,0.384515,NFC,NFC East


In [35]:
# Check for nulls
merged.isnull().sum()

date           0
datetime       0
season         0
team           0
elo            0
probability    0
conference     0
division       0
dtype: int64

In [36]:
merged.to_csv('data/nfl_elo.csv', index=False)